In [ ]:
pip install tensorflow_decision_forests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 59.1 MB/s eta 0:00:00


**1. Data Extraction and data spliting**

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import os  # Import the os module to handle file paths
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/online_shoppers_intention.csv')
df = df.drop_duplicates()
# Initialize the LabelEncoder
label_encoder = LabelEncoder()
# Fit and transform the 'Month' column
df['Month'] = label_encoder.fit_transform(df['Month'])
df['VisitorType'] = label_encoder.fit_transform(df['VisitorType'])
df['Weekend'] = df['Weekend'].astype(int)
df['Revenue'] = df['Revenue'].astype(int)
# Define feature and target columns
X = df.drop(columns=['Revenue'])  # Features
y = df['Revenue']  # Target label
# Train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Convert the training data to a TensorFlow dataset
train_dataset = pd.concat([X_train, y_train], axis=1)
test_dataset = pd.concat([X_test, y_test], axis=1)
# Save the splits as CSV files
train_dataset.to_csv('train_data.csv', index=False)
test_dataset.to_csv('test_data.csv', index=False)
y_test.to_csv('y_test.csv',index = False)

Traing Model and Saving

In [ ]:
import tensorflow as tf
import tensorflow_decision_forests as tfdf
import pandas as pd
import pickle
import os  # Import the os module to handle file paths
#loading the training data
train_dataset = pd.read_csv('train_data.csv')
# Convert pandas DataFrames to TensorFlow datasets
train_tf_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(train_dataset, label="Revenue")
model = tfdf.keras.RandomForestModel()
model.fit(train_tf_dataset)
# Save the model to the 'Models' folder using pickle
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)

Use /tmp/tmp76mx7say as temporary training directory
Reading training dataset...
Training dataset read in 0:00:17.277626. Found 9764 examples.
Training model...
Model trained in 0:00:16.070575
Compiling model...
Model compiled.


#Scoring and Predictions

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import f1_score, precision_score, recall_score
import pandas as pd
import pickle
import os  # Import the os module to handle file paths
#loading the test dataset
test_dataset = pd.read_csv('test_data.csv')
# Convert pandas DataFrames to TensorFlow datasets
test_tf_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(test_dataset, label="Revenue")
y_test = pd.read_csv('y_test.csv')
y_test = y_test['Revenue']
# Load the model from the file
with open('model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)
#evaluation = loaded_model.evaluate(test_tf_dataset)
#print("Random Forest model evaluation:", evaluation)
#Predictions
predictions = model.predict(test_tf_dataset)
# Convert predictions to label format
predicted_classes = [round(pred[0]) for pred in predictions]  # Round the predicted value to nearest class
accuracy = accuracy_score(y_test, predicted_classes)
f1 = f1_score(y_test, predicted_classes, average='weighted')
precision = precision_score(y_test, predicted_classes, average='weighted')
recall = recall_score(y_test, predicted_classes, average='weighted')
# Print the metrics
print(f"Accuracy: {accuracy*100}")
print(f"F1 Score: {f1*100}")
print(f"Precision: {precision*100}")
print(f"Recall: {recall*100}")
# Save predictions to CSV
predictions = pd.DataFrame({'Actual': y_test, 'Predicted': predicted_classes})
predictions.to_csv('predictions.csv', index=False)

Use /tmp/tmp5jkaqh8p as temporary training directory


3/3 [==============================] - 0s 30ms/step
Accuracy: 90.5366653011061
F1 Score: 89.88178222629125
Precision: 89.82128508738602
Recall: 90.5366653011061


In [ ]:
!python run.py


python3: can't open file '/content/run.py': [Errno 2] No such file or directory
